In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv")

In [4]:
df = df.drop("name", axis=1)

In [5]:
df.head()

mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin  
0          70    usa  
1          70    usa  
2          70    usa  
3          70    usa  
4          70    usa

In [7]:
df.origin.value_counts()

origin
usa       249
japan      79
europe     70
Name: count, dtype: int64

In [8]:
def origin(x):
    if x == "usa":
        return 1
    else:
        return 0

In [9]:
df["origin"] = df.origin.apply(origin)

In [10]:
df.head()

mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year  origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1

In [11]:
X = df.drop("mpg", axis=1)

In [12]:
y = df.mpg

In [63]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [14]:
norm = MinMaxScaler()
norm.fit(X)
X_sc = norm.transform(X)

In [17]:
params = {"max_depth": [3,5,7,9], "n_estimators": [100, 200, 300]}
rf = RandomForestRegressor(n_jobs=-1)

grid = RandomizedSearchCV(
    estimator=rf,
    param_distributions=params,
    n_iter=200,
    cv=3,
    n_jobs=-1
)

grid.fit(X_sc,y)


c:\Users\Ort\miniconda3\envs\flask_env\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=200. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1), n_iter=200,
                   n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 7, 9],
                                        'n_estimators': [100, 200, 300]})

In [65]:
pipe = Pipeline([
("norm", MinMaxScaler()),
 ("rf", RandomForestRegressor(n_jobs=-1))
])

params = {"rf__max_depth": [3,5,7,9], "rf__n_estimators": [100, 200, 300]}

grid = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=params,
    n_iter=200,
    cv=3,
    n_jobs=-1
)

grid.fit(X,y)



c:\Users\Ort\miniconda3\envs\flask_env\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=200. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('norm', MinMaxScaler()),
                                             ('rf',
                                              RandomForestRegressor(n_jobs=-1))]),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'rf__max_depth': [3, 5, 7, 9],
                                        'rf__n_estimators': [100, 200, 300]})

In [66]:
grid.best_score_


0.6045941736244133

In [68]:
grid.best_estimator_.score(X, y)

0.9779287448699718

In [69]:
import pickle 
with open("model.pkl", "wb") as f:
    pickle.dump(grid.best_estimator_, f)

In [70]:
with open("model.pkl", "rb") as f:
    saved_model = pickle.load(f)

In [73]:
pred_df = pd.DataFrame(np.array([8,307, 130, 3504,12, 70, 1]).reshape(1,-1))
pred_df

0    1    2     3   4   5  6
0  8  307  130  3504  12  70  1

In [47]:
y.head(1)

0    18.0
Name: mpg, dtype: float64

In [76]:
saved_model.predict(pred_df)[0]

c:\Users\Ort\miniconda3\envs\flask_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


17.244802745500113

In [75]:
saved_model.feature_names_in_

array(['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin'], dtype=object)

In [79]:
from sqlalchemy import create_engine
import sqlite3

In [78]:
churro = "sqlite:///mpgdb.db"

In [80]:
engine = create_engine(churro)

In [87]:
inputs = str(pred_df.values[0])

In [88]:
outputs = saved_model.predict(pred_df)[0]

c:\Users\Ort\miniconda3\envs\flask_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [107]:
date = str(datetime.now())[0:19]
date

'2024-06-10 10:31:55'

In [99]:
log_df = pd.DataFrame({"inputs":[inputs], "outputs": [outputs], "date": [date]})

In [103]:
log_df.to_sql("logs", con=engine, if_exists="append", index=None)

1

In [112]:
pd.read_sql("SELECT * FROM logs", con=engine)

inputs    outputs  \
0           [   8  307  130 3504   12   70    1]  17.244803   
1  ['10', '400', '250', '4000', '15', '70', '1']  13.501718   
2  ['20', '400', '250', '4000', '15', '70', '1']  13.501718   
3  ['20', '400', '250', '4000', '15', '70', '1']  13.501718   

                  date  
0  2024-06-10 10:22:49  
1  2024-06-10 10:34:44  
2  2024-06-10 10:36:00  
3  2024-06-10 10:45:44

In [111]:
import requests

url = "http://127.0.0.1:5000/predict?cylinders=20&displacement=400&horsepower=250&weight=4000&acceleration=15&model_year=70&origin=1"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


13.501718215994533
